## Import into colmap and run reconstruction

In [20]:
import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags
import pycolmap
from database import COLMAPDatabase, image_ids_to_pair_id

def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size
    
    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]
        # print(f'Adding keypoints for {filename}')
        fname_with_ext = filename + img_ext
        # print(fname_with_ext)

        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

In [21]:
feature_dir = 'features'
dirname = 'dirname'


def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

database_path = 'colmap.db'
!rm {database_path}
import_into_colmap(dirname,feature_dir=feature_dir, database_path=database_path)

 29%|██▉       | 1400/4851 [00:00<00:00, 8900.66it/s]


In [22]:
output_path = 'colmap'

In [23]:
pycolmap.match_exhaustive(database_path)

if not os.path.isdir(output_path):
    os.makedirs(output_path)

maps = pycolmap.incremental_mapping(database_path, dirname, output_path)
maps[0].write(output_path)

I20241111 11:30:53.830128 159087 misc.cc:198] 
Exhaustive feature matching
I20241111 11:30:53.830893 159087 feature_matching.cc:231] Matching block [1/2, 1/2]
I20241111 11:30:58.178336 159087 feature_matching.cc:46]  in 4.347s
I20241111 11:30:58.180120 159087 feature_matching.cc:231] Matching block [1/2, 2/2]
I20241111 11:30:58.903213 159087 feature_matching.cc:46]  in 0.723s
I20241111 11:30:58.904012 159087 feature_matching.cc:231] Matching block [2/2, 1/2]
I20241111 11:31:00.395877 159087 feature_matching.cc:46]  in 1.492s
I20241111 11:31:00.397032 159087 feature_matching.cc:231] Matching block [2/2, 2/2]
I20241111 11:31:04.267244 159087 feature_matching.cc:46]  in 3.870s
I20241111 11:31:04.272563 159087 timer.cc:91] Elapsed time: 0.174 [minutes]
I20241111 11:31:04.287910 159129 misc.cc:198] 
Loading database
I20241111 11:31:04.288489 159129 database_cache.cc:54] Loading cameras...
I20241111 11:31:04.288534 159129 database_cache.cc:64]  99 in 0.000s
I20241111 11:31:04.288542 159129 d

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.573371e+03    0.00e+00    5.69e+04   0.00e+00   0.00e+00  1.00e+04        0    5.95e-04    1.19e-03
   1  3.162546e+03   -5.89e+02    5.69e+04   1.65e+02  -2.80e+00  5.00e+03        1    5.98e-04    1.81e-03
   2  2.506926e+03    6.64e+01    4.34e+04   9.21e+01   4.39e-01  4.99e+03        1    9.30e-04    2.75e-03
   3  2.407718e+03    9.92e+01    3.35e+04   8.02e+01   6.61e-01  5.16e+03        1    1.00e-03    3.76e-03
   4  2.342573e+03    6.51e+01    2.86e+04   7.25e+01   6.36e-01  5.27e+03        1    9.08e-04    4.67e-03
   5  2.291433e+03    5.11e+01    2.43e+04   6.51e+01   6.51e-01  5.42e+03        1    8.85e-04    5.57e-03
   6  2.251398e+03    4.00e+01    2.11e+04   5.91e+01   6.51e-01  5.57e+03        1    8.90e-04    6.46e-03
   7  2.218494e+03    3.29e+01    1.83e+04   5.40e+01   6.60e-01  5.76e+03        1    9.51e-04    7.42e-03
   8  2.191116e+03    2.74e+

I20241111 11:31:04.470206 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:31:04.470237 159129 bundle_adjustment.cc:942] 
    Residuals : 1724
   Parameters : 1302
   Iterations : 101
         Time : 0.087162 [s]
 Initial cost : 1.22175 [px]
   Final cost : 1.01668 [px]
  Termination : No convergence

I20241111 11:31:04.470604 159129 incremental_mapper.cc:160] => Filtered observations: 34
I20241111 11:31:04.470613 159129 incremental_mapper.cc:167] => Filtered images: 0
I20241111 11:31:04.560052 159129 misc.cc:198] 
Registering image #99 (3)
I20241111 11:31:04.560073 159129 incremental_mapper.cc:495] => Image sees 112 / 2684 points
I20241111 11:31:04.560163 159129 incremental_mapper.cc:535] => Could not register, trying another image.
I20241111 11:31:04.560169 159129 misc.cc:198] 
Registering image #10 (3)
I20241111 11:31:04.560173 159129 incremental_mapper.cc:495] => Image sees 126 / 3112 points
I20241111 11:31:04.560278 159129 incremental_mapper.cc:5

  27  1.381198e+03    9.85e-01    3.60e+03   1.49e+02   8.96e-01  5.16e+06        1    8.56e-04    2.36e-02
  28  1.381380e+03   -1.82e-01    3.60e+03   2.47e+02  -1.32e-01  2.58e+06        1    3.67e-04    2.40e-02
  29  1.380639e+03    5.59e-01    3.45e+03   1.21e+02   7.20e-01  2.82e+06        1    8.83e-04    2.49e-02
  30  1.380102e+03    5.37e-01    1.56e+03   1.62e+02   6.23e-01  2.86e+06        1    8.53e-04    2.58e-02
  31  1.379664e+03    4.39e-01    3.39e+02   1.17e+02   4.67e-01  2.86e+06        1    8.37e-04    2.66e-02
  32  1.381056e+03   -1.39e+00    3.39e+02   1.78e+02  -1.28e+00  1.43e+06        1    3.43e-04    2.69e-02
  33  1.380719e+03   -1.06e+00    3.39e+02   1.09e+02  -1.26e+00  3.58e+05        1    3.34e-04    2.73e-02
  34  1.380219e+03   -5.55e-01    3.39e+02   4.18e+01  -8.89e-01  4.47e+04        1    3.28e-04    2.76e-02
  35  1.379618e+03    4.58e-02    4.17e+02   7.92e+00   8.70e-02  2.86e+04        1    8.22e-04    2.84e-02
  36  1.379330e+03    2.88e-

I20241111 11:31:04.724871 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:31:04.724920 159129 bundle_adjustment.cc:942] 
    Residuals : 1644
   Parameters : 1242
   Iterations : 101
         Time : 0.0826929 [s]
 Initial cost : 0.892912 [px]
   Final cost : 0.886917 [px]
  Termination : No convergence

I20241111 11:31:04.725405 159129 incremental_mapper.cc:175] => Completed observations: 0
I20241111 11:31:04.725505 159129 incremental_mapper.cc:178] => Merged observations: 0
I20241111 11:31:04.725694 159129 incremental_mapper.cc:160] => Filtered observations: 6
I20241111 11:31:04.725719 159129 incremental_mapper.cc:119] => Changed observations: 0.007299
I20241111 11:31:04.725747 159129 misc.cc:198] 
Global bundle adjustment
I20241111 11:31:04.816442 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:31:04.816483 159129 bundle_adjustment.cc:942] 
    Residuals : 1632
   Parameters : 1233
   Iterations : 101
         Ti

...

I20241111 11:32:52.412847 159129 misc.cc:198] 
Finding good initial image pair
I20241111 11:32:52.577083 159129 misc.cc:198] 
Initializing with image pair #33 and #24
I20241111 11:32:52.577462 159129 misc.cc:198] 
Global bundle adjustment
I20241111 11:32:52.609205 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:32:52.609234 159129 bundle_adjustment.cc:942] 
    Residuals : 584
   Parameters : 447
   Iterations : 101
         Time : 0.0314391 [s]
 Initial cost : 1.45837 [px]
   Final cost : 0.830105 [px]
  Termination : No convergence

I20241111 11:32:52.609417 159129 incremental_mapper.cc:160] => Filtered observations: 2
I20241111 11:32:52.609423 159129 incremental_mapper.cc:167] => Filtered images: 0


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.242080e+03    0.00e+00    2.67e+04   0.00e+00   0.00e+00  1.00e+04        0    2.29e-04    5.35e-04
   1  2.488586e+03   -1.25e+03    2.67e+04   2.77e+02  -3.35e+00  5.00e+03        1    2.64e-04    8.49e-04
   2  1.239136e+03    2.94e+00    3.55e+05   1.87e+02   9.21e-03  2.57e+03        1    3.26e-04    1.18e-03
   3  8.782563e+02    3.61e+02    6.61e+04   6.82e+01   9.69e-01  7.71e+03        1    3.87e-04    1.57e-03
   4  1.080543e+03   -2.02e+02    6.61e+04   1.32e+02  -2.38e+00  3.85e+03        1    1.21e-04    1.70e-03
   5  8.537360e+02    2.45e+01    7.35e+04   7.57e+01   4.32e-01  3.84e+03        1    3.08e-04    2.01e-03
   6  8.043612e+02    4.94e+01    3.97e+04   6.42e+01   7.66e-01  4.52e+03        1    3.19e-04    2.33e-03
   7  7.749625e+02    2.94e+01    2.74e+04   6.43e+01   6.98e-01  4.82e+03        1    3.12e-04    2.65e-03
   8  7.486531e+02    2.63e+

I20241111 11:32:52.719709 159129 misc.cc:198] 
Registering image #35 (3)
I20241111 11:32:52.719739 159129 incremental_mapper.cc:495] => Image sees 33 / 881 points
I20241111 11:32:52.726436 159129 incremental_mapper.cc:535] => Could not register, trying another image.
I20241111 11:32:52.726451 159129 misc.cc:198] 
Retriangulation
I20241111 11:32:52.726528 159129 incremental_mapper.cc:175] => Completed observations: 0
I20241111 11:32:52.726557 159129 incremental_mapper.cc:178] => Merged observations: 0
I20241111 11:32:52.726559 159129 incremental_mapper.cc:105] => Retriangulated observations: 0
I20241111 11:32:52.726645 159129 misc.cc:198] 
Global bundle adjustment
I20241111 11:32:52.758528 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:32:52.758556 159129 bundle_adjustment.cc:942] 
    Residuals : 580
   Parameters : 444
   Iterations : 101
         Time : 0.031642 [s]
 Initial cost : 0.815711 [px]
   Final cost : 0.807617 [px]
  Termination : No con

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.119076e+03    0.00e+00    6.72e+05   0.00e+00   0.00e+00  1.00e+04        0    2.10e-04    4.85e-04
   1  1.631186e+03    4.88e+02    9.46e+03   9.43e+01   9.70e-01  3.00e+04        1    4.45e-04    9.43e-04
   2  1.631293e+03   -1.07e-01    9.46e+03   1.19e+02  -1.26e-03  1.50e+04        1    1.40e-04    1.09e-03
   3  1.580063e+03    5.11e+01    8.46e+03   6.72e+01   7.65e-01  1.76e+04        1    3.31e-04    1.43e-03
   4  1.544436e+03    3.56e+01    4.82e+03   5.64e+01   9.31e-01  4.91e+04        1    3.14e-04    1.74e-03
   5  1.525393e+03    1.90e+01    8.58e+04   1.52e+02   5.02e-01  4.91e+04        1    3.03e-04    2.05e-03
   6  1.484751e+03    4.06e+01    1.13e+05   1.56e+02   7.93e-01  6.14e+04        1    4.23e-04    2.48e-03
   7  1.457095e+03    2.77e+01    1.16e+05   2.01e+02   5.36e-01  6.14e+04        1    3.34e-04    2.82e-03
   8  1.430339e+03    2.68e+

I20241111 11:32:53.228128 159129 misc.cc:198] 
Initializing with image pair #86 and #85
I20241111 11:32:53.228842 159129 misc.cc:198] 
Global bundle adjustment
I20241111 11:32:53.269158 159129 misc.cc:205] 
Bundle adjustment report
------------------------
I20241111 11:32:53.269194 159129 bundle_adjustment.cc:942] 
    Residuals : 624
   Parameters : 477
   Iterations : 101
         Time : 0.0396771 [s]
 Initial cost : 2.28734 [px]
   Final cost : 0.954574 [px]
  Termination : No convergence

I20241111 11:32:53.269471 159129 incremental_mapper.cc:160] => Filtered observations: 14
I20241111 11:32:53.269488 159129 incremental_mapper.cc:167] => Filtered images: 0
I20241111 11:32:53.370505 159129 misc.cc:198] 
Finding good initial image pair
I20241111 11:32:53.371220 159129 incremental_mapper.cc:404] => No good initial image pair found.
I20241111 11:32:53.371335 159129 timer.cc:91] Elapsed time: 1.818 [minutes]


In [24]:
# dense reconstruction
pycolmap.undistort_images(output_path, output_path, dirname)
pycolmap.patch_match_stereo(output_path)  # requires compilation with CUDA
pycolmap.stereo_fusion(output_path / "dense.ply", output_path)

I20241111 11:32:53.403241 64385 images.h:135]  => Reconstruction with 2 images and 408 points
I20241111 11:32:53.403620 162714 misc.cc:198] 
Image undistortion
I20241111 11:32:53.403905 162714 undistortion.cc:210] Undistorting image [1/2]
I20241111 11:32:54.957727 162714 undistortion.cc:210] Undistorting image [2/2]
I20241111 11:32:54.969127 162714 undistortion.cc:223] Writing reconstruction...
I20241111 11:32:55.016654 162714 undistortion.cc:228] Writing configuration...
I20241111 11:32:55.016738 162714 undistortion.cc:232] Writing scripts...
I20241111 11:32:55.016821 162714 timer.cc:91] Elapsed time: 0.027 [minutes]


AttributeError: module 'pycolmap' has no attribute 'patch_match_stereo'

In [ ]:
#model_dir = os.path.abspath('colmap_rec_loftr/0')
#ply_model_path = os.path.abspath('model.ply')
#!colmap model_converter --input_path "{model_dir}" --output_path "{ply_model_path}" --output_type PLY

In [ ]:
#import open3d as o3d

# def visualize_ply(ply_model_path):
#     # Load the .ply file
#     pcd = o3d.io.read_point_cloud(ply_model_path)

#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])

# visualize_ply('model.ply')

In [ ]:
# !colmap model_merger \
#     --input_path1 ./colmap_rec_loftr/0-1 \
#     --input_path2 ./colmap_rec_loftr/2 \
#     --output_path ./colmap_rec_loftr/total


In [25]:
!colmap gui

qt.qpa.xcb: internal error:  void QXcbWindow::setNetWmStateOnUnmappedWindow() called on mapped window
QOpenGLVertexArrayObject::destroy() failed to restore current context
